# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress
from citipy import citipy
from datetime import datetime
import time

# Impor the OpenWeatherMap API key
from config import weather_api_key

# Define URL
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial"
query_url = base_url + "&appid=" + weather_api_key + "&q="

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy



### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)


# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 613


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [3]:
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial"
query_url = base_url + "&appid=" + weather_api_key

In [4]:

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = query_url + "&q=" + city 
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the City information into city_data list
        city_data.append({"city": city, 
                          "lat": city_lat, 
                          "lng": city_lng, 
                          "max_Temp": city_max_temp,
                          "humidity": city_humidity,
                          "cloudiness": city_clouds,
                          "wind_Speed": city_wind,
                          "country": city_country,
                          "date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | port elizabeth
Processing Record 2 of Set 1 | touros
Processing Record 3 of Set 1 | port alfred
Processing Record 4 of Set 1 | thompson
Processing Record 5 of Set 1 | flinders
Processing Record 6 of Set 1 | amapa
Processing Record 7 of Set 1 | busselton
Processing Record 8 of Set 1 | meyungs
City not found. Skipping...
Processing Record 9 of Set 1 | bethel
Processing Record 10 of Set 1 | broome
Processing Record 11 of Set 1 | ravar
Processing Record 12 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 13 of Set 1 | punta arenas
Processing Record 14 of Set 1 | sitka
Processing Record 15 of Set 1 | longyearbyen
Processing Record 16 of Set 1 | olafsvik
Processing Record 17 of Set 1 | lavrentiya
Processing Record 18 of Set 1 | kathmandu
Processing Record 19 of Set 1 | atuona
Processing Record 20 of Set 1 | grand-santi
Processing Record 21 of Set 1 | nikolskoye
Processing Record 22 

In [5]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

city          569
lat           569
lng           569
max_Temp      569
humidity      569
cloudiness    569
wind_Speed    569
country       569
date          569
dtype: int64

In [6]:
# Display sample data
city_data_df.head(10)

,city,lat,lng,max_Temp,humidity,cloudiness,wind_Speed,country,date
0,port elizabeth,-33.9180,25.5701,64.69,81,0,17.27,ZA,2023-04-03 21:52:44
1,touros,-5.1989,-35.4608,79.14,77,80,7.49,BR,2023-04-03 21:55:38
2,port alfred,-33.5906,26.8910,63.18,85,64,7.00,ZA,2023-04-03 21:55:38
3,thompson,55.7435,-97.8558,15.96,38,20,2.30,CA,2023-04-03 21:55:38
4,flinders,-34.5833,150.8552,59.25,70,33,1.54,AU,2023-04-03 21:55:38
5,amapa,1.0000,-52.0000,75.65,83,100,2.84,BR,2023-04-03 21:52:57
6,busselton,-33.6500,115.3333,60.21,82,35,8.41,AU,2023-04-03 21:52:39
7,bethel,41.3712,-73.4140,58.05,50,0,14.97,US,2023-04-03 21:55:39
8,broome,42.2506,-75.8330,62.65,43,81,4.00,US,2023-04-03 21:52:45
9,ravar,31.2656,56.8055,55.54,17,0,1.19,IR,2023-04-03 21:55:39


In [7]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [8]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,city,lat,lng,max_Temp,humidity,cloudiness,wind_Speed,country,date
City_ID,,,,,,,,,
0,port elizabeth,-33.9180,25.5701,64.69,81,0,17.27,ZA,2023-04-03 21:52:44
1,touros,-5.1989,-35.4608,79.14,77,80,7.49,BR,2023-04-03 21:55:38
2,port alfred,-33.5906,26.8910,63.18,85,64,7.00,ZA,2023-04-03 21:55:38
3,thompson,55.7435,-97.8558,15.96,38,20,2.30,CA,2023-04-03 21:55:38
4,flinders,-34.5833,150.8552,59.25,70,33,1.54,AU,2023-04-03 21:55:38


### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [9]:
# Build scatter plot for latitude vs. temperature
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

#### Latitude Vs. Humidity

In [10]:
# Build the scatter plots for latitude vs. humidity
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

#### Latitude Vs. Cloudiness

In [11]:
# Build the scatter plots for latitude vs. cloudiness
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

#### Latitude vs. Wind Speed Plot

In [12]:
# Build the scatter plots for latitude vs. wind speed
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [13]:
# Define a function to create Linear Regression plots
# YOUR CODE HERE

In [14]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
# YOUR CODE HERE

# Display sample data
northern_hemi_df.head()

NameError: name 'northern_hemi_df' is not defined

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
# YOUR CODE HERE

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Linear regression on Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE